In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from PIL import Image
import random

import os
import json
import cv2

import torch
from torch.utils.data import random_split, DataLoader, Dataset
from torchvision import datasets, transforms

from transformers import TrOCRProcessor, VisionEncoderDecoderModel, default_data_collator
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments



Завиксируем всю случайность!

In [2]:
seed = 23
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.determenistic = True

Соберем наш датасет из полученных файлов

In [3]:
basketball_df = pd.read_csv('for_train\\train_bascetball.csv')
streetball_df = pd.read_csv('for_train\\train_streetball.csv')
volleyball_df = pd.read_csv('for_train\\train_volleyball.csv')

res_df = pd.concat([basketball_df, streetball_df, volleyball_df], axis=0)
res_df.shape

(150637, 3)

Разобьем датасет на train test и eval части

In [4]:
diff_df, test_df, = train_test_split(res_df, test_size=0.05, train_size=0.2, shuffle=True)
train_df, eval_df = train_test_split(diff_df, test_size=0.2)
# we reset the indices to start from zero
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)
eval_df.reset_index(drop=True, inplace=True)

In [5]:
store_pathes = []

class IAMDataset(Dataset):
    def __init__(self, root_dir, df, processor, max_target_length=3):
        self.root_dir = root_dir
        self.df = df
        self.processor = processor
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
      
        file_name = self.df['file_name'][idx]
        text = str(self.df['text'][idx])
        store_pathes.append(file_name)
   
        image = Image.open(file_name).convert("RGB")
        image = image.resize((64, 64))
        pixel_values = self.processor(image, return_tensors="pt").pixel_values
        # add labels (input_ids) by encoding the text
        labels = self.processor.tokenizer(text, 
                                          padding="max_length", 
                                          max_length=self.max_target_length).input_ids
        # important: make sure that PAD tokens are ignored by the loss function
        labels = [label if label != self.processor.tokenizer.pad_token_id else -100 for label in labels]

        encoding = {"pixel_values": pixel_values.squeeze(), "labels": torch.tensor(labels)}
        return encoding

Создадим наши датасеты

In [6]:
from transformers import TrOCRProcessor

processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
train_dataset = IAMDataset(root_dir='C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\',
                           df=train_df,
                           processor=processor)
test_dataset = IAMDataset(root_dir='C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\',
                           df=test_df,
                           processor=processor)
eval_dataset = IAMDataset(root_dir='C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\',
                           df=eval_df,
                           processor=processor)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [7]:
print("Number of training examples:", len(train_dataset))
print("Number of testing examples:", len(test_dataset))
print("Number of validation examples:", len(eval_dataset))

Number of training examples: 24101
Number of testing examples: 7532
Number of validation examples: 6026


Загрузим предъобученный трансформер

In [8]:
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-stage1")

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-stage1 and are newly initialized: ['encoder.pooler.dense.weight', 'encoder.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#model = VisionEncoderDecoderModel.from_pretrained('model')
model.to(device)
print(device)

cuda


Сконфигурируем нашу модель

In [10]:
# set special tokens used for creating the decoder_input_ids from the labels
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
# make sure vocab size is set correctly
model.config.vocab_size = model.config.decoder.vocab_size

# set beam search parameters
model.config.eos_token_id = processor.tokenizer.sep_token_id
model.config.max_new_tokens = 4
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 10

Сконфигурируем цикл обучения

In [11]:
training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    num_train_epochs=14,
    evaluation_strategy="steps",
    per_device_train_batch_size=3,
    per_device_eval_batch_size=3,
    fp16=True, 
    output_dir="G:\\models1",
    logging_steps=4000,
    save_steps=40000,
    eval_steps=20000,
)

Оопределим метрику

In [12]:
import evaluate

cer_metric = evaluate.load("cer")

In [13]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(labels_ids, skip_special_tokens=True)

    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return {"cer": cer}

Переопределим оптимизатор

In [14]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters())

Запустим цикл обучения

In [15]:
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=processor.feature_extractor,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=default_data_collator,
)
trainer.train()

model.save_pretrained('model_next')

c:\Users\Mytre\OneDrive\Документы\Data\Work\.env\lib\site-packages\transformers\models\trocr\processing_trocr.py:134: FutureWarning: `feature_extractor` is deprecated and will be removed in v5. Use `image_processor` instead.
  warnings.warn(
c:\Users\Mytre\OneDrive\Документы\Data\Work\.env\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/112476 [00:00<?, ?it/s]

{'loss': 1.4224, 'learning_rate': 4.822362103915502e-05, 'epoch': 0.5}
{'loss': 1.0816, 'learning_rate': 4.644635299975106e-05, 'epoch': 1.0}
{'loss': 0.7955, 'learning_rate': 4.4669084960347096e-05, 'epoch': 1.49}
{'loss': 0.6611, 'learning_rate': 4.289181692094314e-05, 'epoch': 1.99}
{'loss': 0.5489, 'learning_rate': 4.111454888153917e-05, 'epoch': 2.49}


c:\Users\Mytre\OneDrive\Документы\Data\Work\.env\lib\site-packages\transformers\generation\utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


  0%|          | 0/2009 [00:00<?, ?it/s]

...

{'eval_loss': 13.836236953735352, 'eval_cer': 0.28525077734216575, 'eval_runtime': 1081.2766, 'eval_samples_per_second': 5.573, 'eval_steps_per_second': 1.858, 'epoch': 7.47}
{'loss': 0.2424, 'learning_rate': 2.1563266830257122e-05, 'epoch': 7.97}
{'loss': 0.2132, 'learning_rate': 1.978644333013265e-05, 'epoch': 8.46}
{'loss': 0.2112, 'learning_rate': 1.8008730751449197e-05, 'epoch': 8.96}
{'loss': 0.1778, 'learning_rate': 1.6231462712045238e-05, 'epoch': 9.46}
{'loss': 0.1711, 'learning_rate': 1.4453750133361785e-05, 'epoch': 9.96}


  0%|          | 0/2009 [00:00<?, ?it/s]

{'eval_loss': 13.837630271911621, 'eval_cer': 0.21184263890766528, 'eval_runtime': 1086.7712, 'eval_samples_per_second': 5.545, 'eval_steps_per_second': 1.849, 'epoch': 9.96}
{'loss': 0.146, 'learning_rate': 1.2676482093957823e-05, 'epoch': 10.46}
{'loss': 0.1416, 'learning_rate': 1.089876951527437e-05, 'epoch': 10.95}
{'loss': 0.1161, 'learning_rate': 9.121946015149898e-06, 'epoch': 11.45}
{'loss': 0.1116, 'learning_rate': 7.344233436466446e-06, 'epoch': 11.95}
{'loss': 0.1, 'learning_rate': 5.566520857782994e-06, 'epoch': 12.45}


  0%|          | 0/2009 [00:00<?, ?it/s]

{'eval_loss': 12.808854103088379, 'eval_cer': 0.254562660538056, 'eval_runtime': 1091.2667, 'eval_samples_per_second': 5.522, 'eval_steps_per_second': 1.841, 'epoch': 12.45}
{'loss': 0.0891, 'learning_rate': 3.7888082790995413e-06, 'epoch': 12.94}
{'loss': 0.0832, 'learning_rate': 2.0119847789750705e-06, 'epoch': 13.44}
{'loss': 0.0674, 'learning_rate': 2.3427220029161775e-07, 'epoch': 13.94}
{'train_runtime': 46699.1181, 'train_samples_per_second': 7.225, 'train_steps_per_second': 2.409, 'train_loss': 0.3549435450811212, 'epoch': 14.0}


In [ ]:
url = "crops\\8\\ballerTV_137example.jpg"
image = Image.open(url).convert("RGB")

pixel_values = processor(image, return_tensors="pt").pixel_values
generated_ids = model.generate(pixel_values.cuda())

generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(f'Номер на футболке: {generated_text}')

Номер на футболке: 8


Определим eval_dataloader и Загрузим обученную модель

In [17]:
test_dataloader = DataLoader(test_dataset, batch_size=32)
model = VisionEncoderDecoderModel.from_pretrained('model')

# set special tokens used for creating the decoder_input_ids from the labels
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
# make sure vocab size is set correctly
model.config.vocab_size = model.config.decoder.vocab_size

# set beam search parameters
model.config.eos_token_id = processor.tokenizer.sep_token_id
model.config.max_new_tokens = 4
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 10

Посчитаем точность обученной модели на eval датасете

In [18]:
from sklearn.metrics import accuracy_score

torch.cuda.empty_cache()
# Define the device to run the evaluation on
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set the model to evaluation mode
model.eval()
model.to(device)
# Evaluate the model on the eval dataset
diff_acc = []
acc = []
i = 0

with torch.no_grad():
    for batch in tqdm(test_dataloader):
    
        target_text = processor.batch_decode(batch['labels'], skip_special_tokens=True)
        target_text = [int(x) for x in target_text]
           
        pixel_values = batch['pixel_values']
        generated_ids = model.generate(pixel_values.cuda(), max_new_tokens=4)       
       
        # Make a prediction
        generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)

        x = [] 
        for j in generated_text:       
            if j.isdigit():
                x.append(int(j))
            else:
                x.append(1000)        
      
        bach_acc = accuracy_score(target_text, x)
        # Save the true and predicted labels
        diff_acc.append(bach_acc)  
        acc.append(bach_acc)

        if (i % 50  == 0) & (i > 2):
            accuracy = np.mean(diff_acc)
            print("Accuracy:", accuracy) 
        i += 1

    print(f"Total accuracy: {np.mean(acc)}")    

  0%|          | 0/236 [00:00<?, ?it/s]

Accuracy: 0.9172794117647058
Accuracy: 0.9173886138613861
Accuracy: 0.914114238410596
Accuracy: 0.914179104477612
Total accuracy: 0.914179104477612


In [46]:
df10 = pd.read_csv('anno_00new.csv')

eval_dataset = IAMDataset(root_dir='C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\',
                           df=df10,
                           processor=processor)

print("Number of validation examples:", len(eval_dataset))

Number of validation examples: 23894


Посчитаем точность обученной модели на eval датасете

In [21]:
from sklearn.metrics import accuracy_score

# Define the device to run the evaluation on
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set the model to evaluation mode
model.eval()
model.to(device)
# Evaluate the model on the eval dataset
y_true = []
y_pred = []
xx = []
with torch.no_grad():
    for i in tqdm(range(len(test_dataset)), nrows=2):
        url = test_df['file_name'][i]
        xx.append(url)
        target_text = int(test_df['text'][i])
        image = Image.open(url).convert("RGB")   

        pixel_values = processor(image, return_tensors="pt").pixel_values
        generated_ids = model.generate(pixel_values.cuda(), max_new_tokens=3)

        generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

        #pixel_values = batch['pixel_values']
        #generated_ids = model.generate(pixel_values.cuda())       

        # Make a prediction
        #generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)
        x = []        
        if generated_text.isdigit():
            
            x = int(generated_text)
        else:
            x = 1000    
        
        # Save the true and predicted labels
        y_true.append(target_text)
        y_pred.append(x)

        if (i % 400  == 0) & (i > 2):
            accuracy = accuracy_score(y_true, y_pred)
            print("Accuracy:", accuracy) 

print("Accuracy:", accuracy)                   

  0%|          | 0/6026 [00:00<?, ?it/s]

Accuracy: 0.6109725685785536
Accuracy: 0.5892634207240949
Accuracy: 0.5870108243130724
Accuracy: 0.5977514053716427
Accuracy: 0.6016991504247876
Accuracy: 0.597667638483965
Accuracy: 0.6012138521956444
Accuracy: 0.6032489846922836
Accuracy: 0.6064981949458483
Accuracy: 0.6080979755061234
Accuracy: 0.6075891842763008
Accuracy: 0.6123724224119975
Accuracy: 0.6129590463372429
Accuracy: 0.6152472772719157
Accuracy: 0.6128978503582736
Accuracy: 0.6128978503582736
